# Code CIS et CIS(D)

In [13]:
import numpy as np
import psi4
import time
def calc_C(Ca, Cb):
    dim = Ca.shape[0] + Cb.shape[0]
    C = np.zeros((dim, dim))
    for i in range(dim):
        if i%2 == 0:
            C[:Ca.shape[0], i] = Ca[:, i//2]
        else:
            C[Cb.shape[0]:,i] = Cb[:, i//2]
    return C

def calc_excitations(nb_occ, nb_so):
    #a-nocc in {0...nvir-1}
    excitations = []
    for i in range(nb_occ):
        for a in range(nb_occ, nb_so):
            excitations.append((i, a))
    return excitations

def transfo_I(I, C):
    #greatly inspired from a Psi4Numpy tutorial (https://github.com/psi4/psi4numpy)
    A = np.block([[I, np.zeros_like(I)],
                 [np.zeros_like(I), I]])
    
    I_oa = np.block([[A.T, np.zeros_like(A.T)],
                 [np.zeros_like(A.T), A.T]])

    #muliken -> dirac notation and antisymetrization
    I_oa = I_oa.transpose(0, 2, 1, 3) - I_oa.transpose(0, 2, 3, 1)

    I_om = np.einsum('pQRS, pP -> PQRS',
          np.einsum('pqRS, qQ -> pQRS',
          np.einsum('pqrS, rR -> pqRS',
          np.einsum('pqrs, sS -> pqrS', I_oa, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)
    return I_om

def calc_H(nb_occ, nb_virt, E_HF, Iom):
    I2 = np.transpose(Iom, (2, 0, 1, 3))[:nb_occ, nb_occ:, :nb_occ, nb_occ:] #1, 2, 0, 3
    Eps = np.zeros((nb_occ*nb_virt, nb_occ*nb_virt))
    for i in range(nb_occ):
        for a in range(nb_virt):
            Eps[i*nb_virt + a, i*nb_virt + a] = E_HF[a+nb_occ] - E_HF[i]
    Hi = np.reshape(I2, (nb_occ*nb_virt, nb_occ*nb_virt))
    Hcis = Hi + Eps
    return Hcis

def output(Ecis, Excit, Ccis):
    seuil_contrib = 0.1
    #old function to print determinant contributions to the excitation, inspired from a Psi4Numpy tutorial
    #Required to compute Excit which is not required for the computation of the energies
    for i, ex in enumerate(Excit):
        j, b = ex
        #print("Etat ", i, " transition ", j, " -> ", b, " énergie: ", Ecis[i], " Ha", " = ", Ha_to_eV(Ecis[i]), " eV")
        print(("Etat {} transition {} -> {} énergie: %.5f Ha" % Ecis[i]).format(i+1, j, b))
        Contrib = Ccis**2
        for k, c in enumerate(Contrib[:,i]):
            if c >= seuil_contrib:
                p,q = Excit[k]
                print(("Fonction {} -> {} contribuion: %.3f" % c).format(p, q))
                
        #print("Contribution: ", Contrib[:, i]) #On récupère la colonne i
        print(" ")




def calc_CIS(hf_wfn, mints):
    '''psi4.set_memory("3 GB")
    psi4.set_options({'basis':        basis,
                      'scf_type':     'pk',
                      'reference':    ref,
                      'mp2_type':     'conv',
                      'e_convergence': 1e-8,
                      'd_convergence': 1e-8})
    hf_e, hf_wfn = psi4.energy('scf', return_wfn=True)
    mints = psi4.core.MintsHelper(hf_wfn.basisset())'''
    
    nbf = mints.nbf() #Nombre de fonctions de base
    na = hf_wfn.nalpha() #Nombre d'ELECTRONS alpha
    nb = hf_wfn.nbeta()
    nocc = na + nb #Nombre de spin-orbitales occupées
    nso = 2*nbf #Nombre de spin orbitales: deux par fonction de base: psi^alpha et psi^beta
    nvir = nso-nocc #Nombre d'orbitales vacantes/virtuelles
    
    #energies are not ordered by symmetry
    eps_a = np.asarray(hf_wfn.epsilon_a_subset("AO", "ALL"))
    eps_b = np.asarray(hf_wfn.epsilon_b_subset("AO", "ALL"))
    eps = np.append(eps_a, eps_b)
    
    #the coef matrix as an alternation of orthogonal alpha and beta columns
    Ca = np.asarray(hf_wfn.Ca_subset("AO", "ALL"))
    Cb = np.asarray(hf_wfn.Cb_subset("AO", "ALL"))
    #from a Psi4Numpy tutorial, do the same as calc_c(Ca, Cb)
    C = np.block([ [Ca, np.zeros_like(Cb)], [np.zeros_like(Ca), Cb] ])
    C = C[:, eps.argsort()]

    eps = np.sort(eps)
    
    I = mints.ao_eri()
    I_om = transfo_I(I, C)
    H_cis = calc_H(nocc, nvir, eps, I_om)
    return H_cis, I_om, C, eps


In [17]:
ev = 27.2114
cm = 219474.63
psi4.core.clean()

mol = psi4.geometry("""
H 0 0 0
H 0 0 0.7""")
basis = 'aug-cc-pvdz'
ref = 'rhf'
numpy_memory = 2
econv = 1e-8
dconv = 1e-8
psi4.set_memory("4 GB")
psi4.core.set_output_file('output.dat', True)

psi4.set_options({'basis':        basis,
                  'scf_type':     'pk',
                  'reference':    ref,
                  'mp2_type':     'conv',
                  'e_convergence': econv,
                  'd_convergence': dconv})
#psi4.optimize('scf')
t = time.time()
hf_e, hf_wfn = psi4.energy('scf', return_wfn=True)
print("Calcul SCF terminé \nEnergie SCF: ", hf_e)
mints = psi4.core.MintsHelper(hf_wfn.basisset())

nbf = mints.nbf() #Number of basis functions
na = hf_wfn.nalpha() #Number of alpha electrons
nb = hf_wfn.nbeta()
nso = 2*nbf #number of spin-orbitals
nocc = na + nb
nvir = nso-nocc 


H_cis, I_om, C, eps = calc_CIS(hf_wfn, mints)
print("Temps d'execution avant diagonalisation: ", time.time()-t)
ECIS, CCIS = np.linalg.eigh(H_cis) #for hermitian matrix
print("Calcul CIS terminé \nNombre d'excitations: ", len(ECIS))
print("Temps d'execution après diagonalisation: ", time.time()-t)
print("\n------------------------ CIS Excitations Energy ------------------------\n")

for i in range(len(ECIS)):
    print("Etat {}: CIS (Ha): {}".format(i, ECIS[i]))

Calcul SCF terminé 
Energie SCF:  -1.1270104181860292
Temps d'execution avant diagonalisation:  1.0852339267730713
Calcul CIS terminé 
Nombre d'excitations:  68
Temps d'execution après diagonalisation:  1.0852339267730713

------------------------ CIS Excitations Energy ------------------------

Etat 0: CIS (Ha): 0.3887415944554436
Etat 1: CIS (Ha): 0.38874159445544404
Etat 2: CIS (Ha): 0.3887415944554442
Etat 3: CIS (Ha): 0.4519926584714859
Etat 4: CIS (Ha): 0.45199265847148706
Etat 5: CIS (Ha): 0.45199265847148723
Etat 6: CIS (Ha): 0.4803059436218058
Etat 7: CIS (Ha): 0.4919724195661962
Etat 8: CIS (Ha): 0.5100147730473418
Etat 9: CIS (Ha): 0.5100147730473422
Etat 10: CIS (Ha): 0.5100147730473422
Etat 11: CIS (Ha): 0.5100147730473424
Etat 12: CIS (Ha): 0.5100147730473426
Etat 13: CIS (Ha): 0.5100147730473431
Etat 14: CIS (Ha): 0.5509030244890111
Etat 15: CIS (Ha): 0.5509030244890114
Etat 16: CIS (Ha): 0.5509030244890118
Etat 17: CIS (Ha): 0.5909981190006196
Etat 18: CIS (Ha): 0.59099

In [15]:
#calcul CIS(D)
pmin = 0
pmax = 10

B = [[CCIS[j:j+nvir,k+pmin] for j in range(0, nvir*nocc, nvir)] for k in range(pmax-pmin)]

DeltaE = [[[[eps[a]+eps[b]-eps[i]-eps[j] for a in range(nocc, nso)] for b in range(nocc, nso)] for j in range(nocc)] for i in range(nocc)]
A = -np.divide(I_om[:nocc, :nocc, nocc:nso, nocc:nso], DeltaE, where=DeltaE!=0)

N1 = np.einsum('pib,ba -> pia', B, np.einsum('jkbc,jkca->ba', I_om[:nocc, :nocc, nocc:nso, nocc:nso],A))


N2 = np.einsum('pja,ji -> pia', B, np.einsum('jkbc, ikcb -> ji', I_om[:nocc, :nocc, nocc:nso, nocc:nso],A))

N3 = 2*np.einsum('pkc, ikac -> pia', np.einsum('jkbc, pjb -> pkc', I_om[:nocc, :nocc, nocc:nso, nocc:nso], B), A)
Nu = (N1 + N2 + N3)/2
N1, N2, N3 = None, None, None

E2 = np.einsum('pia, pia -> p', B, Nu)
Nu = None #vider la mémoire

U1 = np.einsum('icab, pjc -> pijab', I_om[:nocc, nocc:nso, nocc:nso, nocc:nso], B)
U2 = -np.einsum('jcab, pic -> pijab', I_om[:nocc, nocc:nso, nocc:nso, nocc:nso], B)


U3 = -np.einsum('ijak, pkb -> pijab', I_om[:nocc, :nocc, nocc:nso, :nocc], B)
U4 = np.einsum('ijbk, pka -> pijab', I_om[:nocc, :nocc, nocc:nso, :nocc], B)


U = U1 + U2 + U3 + U4
U1, U2, U3, U4 = None, None, None, None
#print(np.allclose(U[:, :nocc, :nocc, nocc:nso, nocc:nso], U_2))

Deno = np.zeros((pmax-pmin, nocc, nocc, nvir, nvir))
for p in range(pmin, pmax):
    Deno[p-pmin] = DeltaE-ECIS[p]
#Deno[p,i,j,a,b] = DeltaE[i,j,a,b] - ECIS[p]
E1 = -np.sum(np.divide(U**2, Deno), axis=(1,2,3,4))/4
E = E1+E2
print("------------------------ CIS(D) Excitations Energy ------------------------\n")

for i in range(len(E)):
    print("Etat {}: CIS (Ha): {}, CIS(D) (Ha): {}\n".format(i+pmin, ECIS[i+pmin], (ECIS[i+pmin]*ev+E[i])))

------------------------ CIS(D) Excitations Energy ------------------------

Etat 0: CIS (Ha): 0.3887415944554436, CIS(D) (Ha): 10.594244471483883

Etat 1: CIS (Ha): 0.38874159445544404, CIS(D) (Ha): 10.594244471483895

Etat 2: CIS (Ha): 0.3887415944554442, CIS(D) (Ha): 10.594244471483899

Etat 3: CIS (Ha): 0.4519926584714859, CIS(D) (Ha): 12.311590012354111

Etat 4: CIS (Ha): 0.45199265847148706, CIS(D) (Ha): 12.311590012354142

Etat 5: CIS (Ha): 0.45199265847148723, CIS(D) (Ha): 12.311590012354147

Etat 6: CIS (Ha): 0.4803059436218058, CIS(D) (Ha): 13.068769058076954

Etat 7: CIS (Ha): 0.4919724195661962, CIS(D) (Ha): 13.38763362670334

Etat 8: CIS (Ha): 0.5100147730473418, CIS(D) (Ha): 13.894974340016226

Etat 9: CIS (Ha): 0.5100147730473422, CIS(D) (Ha): 13.894974340016239



## Première version du code CIS(D), à l'aide de boucles

Ce code ayant été validé à l'aide de Gaussian, il a également servi de références pour vérifier les futures versions du code.

Afin de comparer 2 implémentations pour la création d'un même tableau, la méthode np.allclose a été utilisée.

In [21]:
def calc_excitations(nb_occ, nb_so):
    #a-nocc in {0...nvir-1}
    excitations = []
    for i in range(nb_occ):
        for a in range(nb_occ, nb_so):
            excitations.append((i, a))
    return excitations

t0 = time.time()
hf_e, hf_wfn = psi4.energy('scf', return_wfn=True)
mints = psi4.core.MintsHelper(hf_wfn.basisset())

nbf = mints.nbf() #Nombre de fonctions de base
na = hf_wfn.nalpha() #Nombre d'ELECTRONS alpha
nb = hf_wfn.nbeta()
nocc = na + nb #Nombre de spin-orbitales occupées
nso = 2*nbf #Nombre de spin orbitales: deux par fonction de base: psi^alpha et psi^beta
nvir = nso-nocc #Nombre d'orbitales vacantes/virtuelles

#Pour éviter les problèmes de symétries, on récupère toutes les orbitales (sans les classer par symétrie)
eps_a = np.asarray(hf_wfn.epsilon_a_subset("AO", "ALL"))
eps_b = np.asarray(hf_wfn.epsilon_b_subset("AO", "ALL"))
eps = np.sort(np.append(eps_a, eps_b))


#On def la matrice des coef comme une alternance de colonnes orthogonales alpha et beta
Ca = np.asarray(hf_wfn.Ca_subset("AO", "ALL"))
Cb = np.asarray(hf_wfn.Cb_subset("AO", "ALL"))
C = calc_C(Ca, Cb)

Excitations = calc_excitations(nocc, nso)

I = mints.ao_eri()
I_om = transfo_I(I, C)
print("Nombre d'excitations: ", len(Excitations))
H_cis, I_om, C, eps = calc_CIS(hf_wfn, mints)
ECIS, CCIS = np.linalg.eigh(H_cis) #for hermitian matrix#ind = list(ECIS).index(np.min(ECIS))
CCIS = CCIS[:, ECIS.argsort()]
ECIS = np.sort(ECIS)
def calc_a(i,j,a,b):
    num = -I_om[i,j,a,b]
    den = eps[a]+eps[b]-eps[i]-eps[j]
    return num/den
def calc_omega_cisd(p):
    print("Excitation: ", Excitations[p])
    omega = ECIS[p]
    #Calcul du premier terme
    E1 = 0
    for i in range(nocc):
        for j in range(nocc):
            for a in range(nocc, nso):
                for b in range(nocc, nso):
                    u=0
                    for c in range(nocc, nso):
                        kj = Excitations.index((j,c))
                        ki = Excitations.index((i,c))
                        u+= I_om[i,c,a,b]*CCIS[kj, p] - I_om[j,c,a,b]*CCIS[ki, p]
                    for k in range(nocc):
                        kb = Excitations.index((k,b))
                        ka = Excitations.index((k,a))
                        u += -I_om[i,j,a,k]*CCIS[kb, p] + I_om[i,j,b,k]*CCIS[ka, p]
                    E1 += -(u**2)/(eps[a]+eps[b]-eps[i]-eps[j]-omega)
    E1 = E1/4
    print(E1)
    E2 = 0
    for i in range(nocc):
        print(i)
        for a in range(nocc, nso):
            kia = Excitations.index((i,a))
            nu = 0
            for j in range(nocc):
                for k in range(nocc):
                    for b in range(nocc, nso):
                        for c in range(nocc, nso):
                            k1 = Excitations.index((i,b))
                            k2 = Excitations.index((j,a))
                            k3 = Excitations.index((j,b))
                            a1 = calc_a(j,k,c,a)
                            a2 = calc_a(i,k,c,b)
                            a3 = calc_a(i,k,a,c)
                            nu += (I_om[j,k,b,c])*(CCIS[k1,p]*a1 + CCIS[k2,p]*a2 + 2*CCIS[k3,p]*a3)
            nu = nu/2
            E2 += CCIS[kia,p]*nu
    return E1, E2
ind = 0
E1, E2 = calc_omega_cisd(ind)
print("Temps: ", time.time()-t0)
print("Correction énergétique: ", E1+E2)
print("Energie d'excitation (CIS): ", ECIS[ind])
print("Energie corrigée CIS(D): ", ECIS[ind] + E1 + E2)

Nombre d'excitations:  68
Excitation:  (0, 2)
-0.010862305539213376
0
1
Temps:  9.765986204147339
Correction énergétique:  0.016041448119023353
Energie d'excitation (CIS):  0.3887415944554436
Energie corrigée CIS(D):  0.40478304257446696
